In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
import seaborn as sns
import nltk
import re
import string
import warnings
import emoji
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier


warnings.filterwarnings('ignore')

In [2]:
# Download necessary NLTK data files

# Download stopwords: A list of common words (e.g., "the", "is", "and") that are often removed during text preprocessing.
# These words are typically filtered out because they do not contribute much to the meaning of the text.
nltk.download('stopwords')

# Download punkt: A pre-trained tokenizer model used to split text into sentences or words.
# This is essential for tasks like sentence segmentation and word tokenization.
nltk.download('punkt')

# Download wordnet: A lexical database for the English language, used for tasks like lemmatization and synonym finding.
# WordNet helps in understanding the semantic relationships between words.
nltk.download('wordnet')

# Download omw-1.4: Open Multilingual WordNet, which extends WordNet to support multiple languages.
# This is useful for working with text in languages other than English.
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [3]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [4]:
!pip install symspellpy
!wget https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt -O frequency_dictionary_en.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 60.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 9.1 MB/s eta 0:00:00
--2025-03-06 17:59:08--  https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234910 (4.0M) [text/plain]
Saving to: ‘frequency_dictionary_en.txt’

frequency_dictionar 100%[===================>]   4.04M  --.-KB/s    in 0.02s   

2025-03-06 17:59:09 (203 MB/s) - ‘frequency_dictionary_en.txt’ saved [4234910/4234910]



In [52]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [53]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [54]:
df = df.drop(columns=["keyword", "location", "id"])

In [8]:
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [55]:
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load dictionary
sym_spell.load_dictionary("frequency_dictionary_en.txt", term_index=0, count_index=1)


True

In [56]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])
    
    # Remove extra whitespace
    text = ' '.join(text.split())

    # Removing emojis
    text = emoji.replace_emoji(text, replace='')

    # **Apply SymSpell for spelling correction**
    corrected_words = []
    for word in text.split():
        suggestion = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected_words.append(suggestion[0].term if suggestion else word)

    text = ' '.join(corrected_words)

    # Tokenization using NLTK
    tokens = word_tokenize(text)  

    # Lemmatization
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    
    return ' '.join(tokens)

In [57]:
df['text'] = df['text'].apply(preprocess_text)

In [12]:
df['text']

0              deed reason earthquake may allah forgive u
1                   forest fire near la ronge sask canada
2       resident asked shelter place notified officer ...
3       people receive wildfire evacuation order calif...
4       got sent photo ruby alaska smoke wildfire pour...
                              ...                        
7608    two giant crane holding bridge collapse nearby...
7609    ariaahrary thetawniest control wild fire calif...
7610                                 utckm volcano hawaii
7611    police investigating ebike collided car little...
7612    latest home razed northern california wildfire...
Name: text, Length: 7613, dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7613 non-null   object
 1   target  7613 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.1+ KB


In [69]:
X = df[['text']]
y = df['target']

In [59]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [70]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [17]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [63]:
X

,text
0,deed reason earthquake may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked shelter place notified officer ...
3,people receive wildfire evacuation order calif...
4,got sent photo ruby alaska smoke wildfire pour...
...,...
7608,two giant crane holding bridge collapse nearby...
7609,ariaahrary thetawniest control wild fire calif...
7610,utckm volcano hawaii
7611,police investigating ebike collided car little...


In [19]:
# Combine all reviews into a single text
all_text = ' '.join(df['text'])

# Tokenize the text
all_words = all_text.split()

# Get the total number of words
total_words = len(all_words)

# Get the total number of unique words (vocabulary size)
unique_words = len(set(all_words))

print(f"Total number of words in the corpus: {total_words}")
print(f"Total number of unique words (vocabulary size): {unique_words}")

Total number of words in the corpus: 69503
Total number of unique words (vocabulary size): 15465


In [62]:
X_train.head()

,text
1163,sioux city fire official believe bridge collap...
4989,strongly support military amp family cock suck...
6913,lucysforsale funny cause dumb as young one get...
6711,usnwsgov special weather statement issued augu...
2736,abcnews obama declares disaster typhoondevasta...


In [71]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_features=8000, min_df=2, max_df=0.95)

X_train_bow = vectorizer.fit_transform(X['text']).toarray()
# X_test_bow = vectorizer.transform(X_test['text']).toarray()

In [65]:
# Logistic Regression
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_train_bow, y_train)
y_pred = log_reg.predict(X_test_bow)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)

# Multinomial Naive Bayes (Best for NLP)
nb = MultinomialNB()
nb.fit(X_train_bow, y_train)
y_pred = nb.predict(X_test_bow)
print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)

ValueError: Found input variables with inconsistent numbers of samples: [7613, 6090]

In [73]:
nb = MultinomialNB()
nb.fit(X_train_bow, y)

MultinomialNB()

In [23]:
# svm_clf = SVC() #Initialize SVM classifier
# svm_clf.fit(X_train_bow, y_train) #Train the SVM
# y_pred = svm_clf.predict(X_test_bow) #Make predictions
# print("SVM Accuracy:", accuracy_score(y_test, y_pred)) #Print accuracy
# f1 = f1_score(y_test, y_pred, average='binary') 
# print("F1 Score:", f1)


In [24]:
cv = CountVectorizer(ngram_range=(1,3), max_features=8000)

X_train_bow = cv.fit_transform(X_train['text']).toarray()
X_test_bow = cv.transform(X_test['text']).toarray()

In [25]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train_bow, y_train)
y_pred = log_reg.predict(X_test_bow)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)

# Multinomial Naive Bayes (Best for NLP)
nb = MultinomialNB()
nb.fit(X_train_bow, y_train)
y_pred = nb.predict(X_test_bow)
print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)

Logistic Regression Accuracy: 0.7898883782009193
F1 Score: 0.7288135593220337
Multinomial Naive Bayes Accuracy: 0.8168089297439265
F1 Score: 0.7554776511831727


In [26]:
# svm_clf = SVC(class_weight='balanced', probability=True) #Initialize SVM classifier
# svm_clf.fit(X_train_bow, y_train) #Train the SVM
# y_pred = svm_clf.predict(X_test_bow) #Make predictions
# print("SVM Accuracy:", accuracy_score(y_test, y_pred)) #Print accuracy
# f1 = f1_score(y_test, y_pred, average='binary') 
# print("F1 Score:", f1)

In [27]:
# param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear']}
# grid_search = GridSearchCV(SVC(class_weight='balanced', probability=True), param_grid, scoring='f1')
# grid_search.fit(X_train_bow, y_train)
# best_model = grid_search.best_estimator_

# svm = SVC(kernel='linear', C=1, class_weight='balanced', probability=True)
# svm.fit(X_train_bow, y_train)

# y_probs = svm.predict_proba(X_test_bow)[:, 1]
# threshold = 0.4  # Tune threshold to balance precision & recall
# y_pred = (y_probs > threshold).astype(int)

# print("SVM Accuracy:", accuracy_score(y_test, y_pred))
# f1 = f1_score(y_test, y_pred, average='binary') 
# print("F1 Score:", f1)

In [28]:
tdf= pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [29]:
tdf.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [30]:
tdf = tdf.drop(columns=["keyword", "location"])

In [31]:
tdf['text'] = tdf['text'].apply(preprocess_text)

In [32]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3263 non-null   int64 
 1   text    3263 non-null   object
dtypes: int64(1), object(1)
memory usage: 51.1+ KB


In [74]:
# cv = CountVectorizer(ngram_range=(1,3), max_features=8000)

X_test_final = vectorizer.transform(tdf['text']).toarray()

In [75]:
# y_pred = nb.predict(tdf['text'])
# Predict using the trained Naïve Bayes model
tdf['target'] = nb.predict(X_test_final)

# print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))

In [77]:
submission = tdf[['id', 'target']]  # Keep only ID and predictions
submission.to_csv('submission_1.csv', index=False)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=8000)

X_train_tfidf = tfidf.fit_transform(X_train['text']).toarray()
X_test_tfidf = tfidf.transform(X_test['text']).toarray()

In [37]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train_tfidf, y_train)
y_pred = log_reg.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)

# Multinomial Naive Bayes (Best for NLP)
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
y_pred = nb.predict(X_test_tfidf)
print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='binary') 
print("F1 Score:", f1)


Logistic Regression Accuracy: 0.8010505581089954
F1 Score: 0.7408041060735672
Multinomial Naive Bayes Accuracy: 0.8063033486539725
F1 Score: 0.72910927456382


In [38]:
# svm_clf = SVC() #Initialize SVM classifier
# svm_clf.fit(X_train_tfidf, y_train) #Train the SVM
# y_pred = svm_clf.predict(X_test_tfidf) #Make predictions
# print("SVM Accuracy:", accuracy_score(y_test, y_pred)) #Print accuracy
# f1 = f1_score(y_test, y_pred, average='binary') 
# print("F1 Score:", f1)



In [39]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10]}
# grid_search = GridSearchCV(DecisionTreeClassifier(class_weight='balanced'), param_grid, scoring='f1')
# grid_search.fit(X_train_tfidf, y_train)
# best_model = grid_search.best_estimator_


# dt = DecisionTreeClassifier(class_weight='balanced', max_depth=20, min_samples_split=5)
# dt.fit(X_train_tfidf, y_train)

# y_probs = dt.predict_proba(X_test_tfidf)[:, 1]
# threshold = 0.4  # Lowering threshold increases recall
# y_pred = (y_probs > threshold).astype(int)

# print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
# f1 = f1_score(y_test, y_pred, average='binary') 
# print("F1 Score:", f1)

In [40]:
# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(class_weight='balanced_subsample', n_estimators=100, max_depth=20, min_samples_split=5, random_state=42)
# rf.fit(X_train_tfidf, y_train)

# from imblearn.over_sampling import SMOTE

# smote = SMOTE()
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)
# rf.fit(X_train_resampled, y_train_resampled)

# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [10, 20, 30],
#     'min_samples_split': [2, 5, 10]
# }

# grid_search = GridSearchCV(RandomForestClassifier(class_weight='balanced_subsample'), param_grid, scoring='f1')
# grid_search.fit(X_train_tfidf, y_train)
# best_model = grid_search.best_estimator_


# y_probs = rf.predict_proba(X_test_tfidf)[:, 1]  # Get probability for positive class
# threshold = 0.5  # Adjust based on F1-score optimization
# y_pred = (y_probs > threshold).astype(int)

# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='binary')

# print("Random Forest Accuracy:", accuracy)
# print("Random Forest F1 Score:", f1)

In [41]:
# from sklearn.ensemble import VotingClassifier

# # Initialize individual models
# log_reg = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
# nb = MultinomialNB()

# # Create a Voting Classifier with soft voting
# ensemble = VotingClassifier(
#     estimators=[('lr', log_reg), ('nb', nb)],
#     voting='soft'  # Average predicted probabilities
# )

In [42]:
# ensemble.fit(X_train_bow, y_train)
# y_pred_ensemble = ensemble.predict(X_test_bow)
# print("Ensemble Model Accuracy:", accuracy_score(y_test, y_pred_ensemble))
# f1 = f1_score(y_test, y_pred_ensemble, average='binary') 
# print("F1 Score:", f1)